# SMM for LCM Topics Class
Consider the following DPG:
$$ Y_i = X_i'\beta + e_i$$
where $\beta\in\mathbb{R}^k$ and we have $E[e_i|X_i]=0$.

From the mean independence assumption we get the following identification result:
$$ E[e_iX_i] = 0 \\ E[(Y_i-X_i'\beta)X_i] = 0 \\ E[Y_iX_i] = E[X_iX_i']\beta$$
and hence $\beta$ is identified as long as $E[X_iX_i']$ is invertible:
$$ \beta = E[X_iX_i']^{-1}E[X_iY_i].$$

Now assume we didn't know about this last result.

Usualy GMM would use the moment condition $E[e_iX_i]=0$ and be the solution to the following sample analogue problem:
$$ \hat{\beta}_{GMM} \equiv \argmin_{b\in\mathbb{R}^k} \sum_{i=1}^nx_i(y_i - x_i'b)W_n \sum_{i=1}^n(y_i - x_i'b)x_i'$$
where $W_n$ is a $k\times k$ weight matrix.

Now these can be solved exactly and we have $\hat{\beta}_{GMM} = \hat{\beta}_{OLS}$.

## SMM Approach
Now instead of solving these moment conditions directly, we instead fix some value $\beta_0$ and
- draw the unobserved component of the model, namely $e$ and compute $y$
  - we draw these once at the beginning to not confound optimization over $\beta$ with the drawing of $e$
  - I use a normal distribution for $e$ but my guess is it doesn't really matter because of consistency (Does this depend on `n` or `reps` though?)
- estimate the implied simulated moments, namely $E[Y_iX_i]$ by $\hat{xy} \equiv n^{-1}\sum_{i=1}^ny_ix_i$
- repeat this $R$ times and average over these estimated moments
- evaluate a loss function (e.g. quadratic loss): $||\hat{\overline{xy}} - \hat{xy}_{observed}||_2$.

This loss is a function of $\beta_0$ which we used to generate the simulated data/simulated moment. 
Thus, the goal becomes to minimize this criterion as a function of $\beta_0$ by repeatedly doing the above for different $\beta_0$.
For example, we could do this for a grid of values $\Beta$ and then just take the point $\beta\in \Beta$ with the lowest criterion.
Clearly, this grid search is inefficient and suffers from a curse of dimensionality.

In practice, we will rely on routines used by `estimagic`. This means we will pass some function `criterion` to `estimagic.minimize` where that function only depends on $\beta$.
Tthe minimizer then optimizes this function over $\beta$. The returned value is our solution $\hat{\beta}_{SMM}$ and we can compare it to the analytically computed $\hat{\beta}_{OLS}$.

In [28]:
# Simulated methods of moments for LCM topics class
# import estimagic
from numpy.random import default_rng
import numpy as np
import plotly.express as px
import pandas as pd
from pathlib import Path

import estimagic as em

from smm import smm

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"


In [29]:
this_dir = Path(".")
this_dir = this_dir.resolve()
root = this_dir
print(root)
bld = root / "bld"
src = root / "src"

bld.mkdir(parents=True, exist_ok=True)


C:\Users\budde\projects\lcm\lcm_smm


In [30]:
np.random.seed(123)
reps = 1000
initial_value = np.array([1, 1, 1])
data = pd.read_feather(src / "data.feather")

res = smm(reps, data, initial_value)
print(res)


Minimize with 3 free parameters terminated successfully after 55 criterion evaluations, 55 derivative evaluations and 38 iterations.

The value of criterion improved from 230152513.91493684 to 1.9541359076876184e-06.

The scipy_lbfgsb algorithm reported: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL

Independent of the convergence criteria used by scipy_lbfgsb, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change  3.769e-11***  3.769e-11***
relative_params_change      3.44e-12***  3.441e-12***
absolute_criterion_change  3.769e-12***  3.769e-12***
absolute_params_change     9.133e-12***  9.133e-12***

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)


c:\Users\budde\miniforge3\envs\epp\Lib\site-packages\estimagic\optimization\optimize_result.py:131: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



In [31]:

fig = em.criterion_plot(res, max_evaluations=300)

fig.show()


In [32]:

fig = em.params_plot(
    res,
    max_evaluations=300,
)
fig.show()


In [33]:
beta_smm = res.params

def _multivariate_ols(y, x):
    """Multivariate OLS regression."""
    return np.linalg.inv(x.T @ x) @ x.T @ y

beta_ols = _multivariate_ols(data["Y"], data[["A", "B", "C"]])

print(f"Analytic OLS: {list(beta_ols)}")
print(f"SMM: {list(beta_smm)}")
print(f"Diff: {list(beta_ols - beta_smm)}")


Analytic OLS: [2.8927337573355048, 194.25920102316675, -1.803510381105907]
SMM: [2.8928048726557325, 194.2594787083691, -1.8030959120274355]
Diff: [-7.11153202277437e-05, -0.00027768520234872085, -0.00041446907847153547]
